# BD

In [ ]:
import pandas as pd
import numpy as np
import random
n=100000
def salaire_brut_mensuel(statut):
    # Dictionnaire contenant les marges de salaire selon le statut
    marges_salaire = {
        "Admi / Tech art 36": (1414.37, 3279.44),
        "Employé": (945.96, 3744.35),
        "Agent de maitrise": (2545.20, 4247.14),
        "Cadre": (2496.48, 8896.26),
        "Cadre DNR": (7155.24, 7155.24),
        "Collaborateur 4/4bis": (4006.41, 6112.24),
        "Ouvrier": (1326.81, 3890.00),
        "Executive": (7400.00, 9980.00),
        "Apprenti (B.C)": (1413.54, 1413.54),
        "Apprenti (W.C)": (1077.82, 1766.92),
        "Stagiaire": (600, 1500),
        "Contrat pro (Cadre)":(1077.82, 1766.92),
        "Admi / Tech":(1077.82, 1766.92),
    }

    # Vérifier si le statut est valide
    if statut in marges_salaire:
        min_salaire, max_salaire = marges_salaire[statut]
        if min_salaire == max_salaire:
            return min_salaire
        else:
            
            return round(random.uniform(min_salaire, max_salaire), 2)
    else:
        raise ValueError(f"Statut non reconnu: {statut}")

frontalier = ['Frontaliers', 'Non']
Region = ['Moselle', 'Autre']

statuts_exoneres = ['Stagiaire', 'Apprenti (B.C)', 'Apprenti (W.C)', 'Mandat Social']

def generate_numero_serie():
    return ''.join([str(random.randint(0, 9)) for _ in range(14)])


# Paramètres du SMIC pour le calcul du plafond
smic_horaire = 11.65
heures_par_jour = 7
smic_horaire_taux_specifique = 11.65  # SMIC horaire pour le calcul des taux spécifiques
heures_hebdomadaires = 35
heures_mensuelles = (35 * 52) / 12
plafond_coef = 2.5
semaines_par_an = 52
jours_legal_mois = 23

# Définir les constantes pour CSG-CRDS
PLAFOND_ANNUEL_SS = 46368
PLAFOND_MENSUEL_SS = 3864


ABATTEMENT = 0.0175
# 'Contrat pro (Cadre)','Admi / Tech'
statut_emp = ['Employé' ,'Collaborateur 4/4bis' ,'Admi / Tech art 36', 'Cadre' ,'Cadre DNR' ,'Executive' ,'Apprenti (B.C)', 'Apprenti (W.C)','Ouvrier', 'Agent de maitrise','Stagiaire']
type_contrat_ = ['CDI Standard', 'CDD standard','CDD Ctr pro','CDD Doctorant','Apprenti','Mandat Social','Stagiaire']
Catégorie_texte = ['Employé', 'Expatrié']
Societe = ['Anoflexe','Tiers' ,'Continental France' ,'Contitech France','Continental Digital Services France']


def determiner_taux_pole_demploi(tranche_a,tranche_b):
  """Calcul du taux pole_demploi """
  if   tranche_a == 0.0 :
    taux_7C00 = 0.0
  else :
    taux_7C00 = 4.05
  if   tranche_b == 0.0 :
    taux_7C10 = 0.0
  else :
    taux_7C10 = 4.05


  return taux_7C00,taux_7C10


def calculate_base_pole_demploi(brut_cum,plafond_cum,cum_base_tranche_a_prec,cum_base_tranche_b_prec,type_contrat):
  """Calcul de la base pole_demploi """
  if  type_contrat != 'Mandat Social' and type_contrat !='Stagiaire' and type_contrat !='Apprenti'  :

    if brut_cum < plafond_cum:
      tranche_a = brut_cum - cum_base_tranche_a_prec -cum_base_tranche_b_prec
      if cum_base_tranche_b_prec !=0:
        tranche_b = -cum_base_tranche_b_prec
      else : tranche_b =0
    else:
      tranche_a = plafond_cum - cum_base_tranche_a_prec
      if brut_cum > 4* plafond_cum:
        tranche_b = 4*plafond_cum -cum_base_tranche_a_prec -cum_base_tranche_b_prec
      else :
        tranche_b = brut_cum -cum_base_tranche_a_prec- cum_base_tranche_b_prec

  else :
    tranche_a = 0.0
    tranche_b = 0.0


  return round(tranche_a, 2),round(tranche_b, 2)
def calculate_base_AGS(brut_cum, plafond_cum, cum_base_tranche_a_prec, cum_base_tranche_b_prec, type_contrat):
    # print(f"Inputs: brut_cum={brut_cum}, plafond_cum={plafond_cum}, cum_base_tranche_a_prec={cum_base_tranche_a_prec}, cum_base_tranche_b_prec={cum_base_tranche_b_prec}, type_contrat={type_contrat}")

    if type_contrat not in ['Mandat Social', 'Stagiaire']:
        # Initialisation des variables
        tranche_a = 0.0
        tranche_b = 0.0
        # print("Calcul pour un contrat standard")

        # Cas où le brut cumulé est inférieur au plafond
        if brut_cum < plafond_cum:
            tranche_a = brut_cum - cum_base_tranche_a_prec - cum_base_tranche_b_prec
            # print(f"tranche_a calculée: {tranche_a}")
            tranche_b = -cum_base_tranche_b_prec if cum_base_tranche_b_prec != 0 else 0.0
            # print(f"tranche_b calculée: {tranche_b}")
        else:  # Cas où le brut cumulé est supérieur ou égal au plafond
            tranche_a = plafond_cum - cum_base_tranche_a_prec
            # print(f"tranche_a calculée: {tranche_a}")
            if brut_cum > 4 * plafond_cum:
                tranche_b = 4 * plafond_cum - cum_base_tranche_a_prec - cum_base_tranche_b_prec
            else:
                tranche_b = brut_cum - cum_base_tranche_a_prec - cum_base_tranche_b_prec
            # print(f"tranche_b calculée: {tranche_b}")

        # Calcul de la base AGS
        ags_base = tranche_a + tranche_b
        # print(f"ags_base calculée: {ags_base}")
    else:  # Cas des contrats "Mandat Social" ou "Stagiaire"
        # print("Contrat non éligible")
        tranche_a = 0.0
        tranche_b = 0.0
        ags_base = 0.0

    # Retourner la valeur arrondie
    ags_base = round(ags_base, 2)
    # print(f"Resultat final: {ags_base}")
    return ags_base



def determiner_taux_ags(ags_base):
   """Calcul de la base ags """
   if ags_base == 0.0    :
    taux_ags = 0.0
   else :
    taux_ags = 0.25
   return taux_ags

# Function to calculate the adjusted social security ceiling based on absence days and extra hours
def calculer_plafond_sec_social(jours_absence):
    return PLAFOND_MENSUEL_SS * (jours_travailles_mois / 30)


# Générer les données pour plusieurs mois
np.random.seed(42)
data = []
cumuls_employes = {}
nombre_mois_simules = 12
effectif = np.random.randint(1, 49, n)  # Effectif aléatoire entre 1 et 99



for i in range(n):
    Numero_serie = generate_numero_serie()
    # Utiliser la fonction salaire_brut_mensuel pour définir le brut_mensuel
    statut = np.random.choice(statut_emp)
    type_contrat = np.random.choice(type_contrat_)
    brut_mensuel= salaire_brut_mensuel(statut)
    remuneration_annuelle = round(brut_mensuel * 12, 2)
    #residence = np.random.choice(Catégorie_texte)
    frontaliers = np.random.choice(frontalier)
    regions = np.random.choice(Region)

    # Initialiser les cumuls pour chaque employé
    if Numero_serie not in cumuls_employes:
        cumuls_employes[Numero_serie] = {

           'sum_remunerations': [0] * nombre_mois_simules,
            'sum_smic': [0] * nombre_mois_simules,
            'sum_assiette_plafonnee': [0] * nombre_mois_simules,
            'sum_pass_ajusted':[0] * nombre_mois_simules,


        }


    # Générer une date d'embauche aléatoire en 2024 avant la date de paie
    date_embauche_jour = np.random.randint(1, 31)
    date_embauche_mois = np.random.randint(1, 13)
    while date_embauche_mois > nombre_mois_simules:
        date_embauche_mois = np.random.randint(1, 13)



    cum_base_tranche_a_prec_m1 = 0
    cum_base_tranche_b_prec_m1 = 0




    for mois in range(date_embauche_mois, nombre_mois_simules + 1):
        date_paie = f"2024-{mois:02d}-01"  # Date fictive pour le bulletin de paie

        # Gestion des absences et des heures sups pour chaque mois
        jours_absence = np.random.randint(0, 4)  # Jours d'absence entre 0 et 10
        heures_absence = jours_absence * heures_par_jour
        heures_supplementaires = np.random.randint(0, 12)

        jours_travailles_mois = 30 - date_embauche_jour + 1 - jours_absence
        salaire_absence = round(brut_mensuel * (heures_absence / heures_mensuelles), 2)
        brut_mensuel_apres_absence = round(brut_mensuel - salaire_absence, 2)
        plafond_adjusted = calculer_plafond_sec_social(jours_absence)






        # Calcul du SMIC mensuel
        smic_mensuel = round((heures_hebdomadaires * semaines_par_an / 12) * smic_horaire_taux_specifique, 2)

        # Calcul du SMIC de référence
        smic_reference = round(smic_mensuel * (brut_mensuel_apres_absence / brut_mensuel)  + (heures_supplementaires * smic_horaire), 2)

        plafond_mensuel_reference = round(plafond_coef * smic_reference, 2)

        # Mise à jour des valeurs cumulées de manière synthétique
        if mois > date_embauche_mois:
            cumuls_employes[Numero_serie]['sum_remunerations'][mois - 1] = cumuls_employes[Numero_serie]['sum_remunerations'][mois - 2] + brut_mensuel_apres_absence
            cumuls_employes[Numero_serie]['sum_smic'][mois - 1] = cumuls_employes[Numero_serie]['sum_smic'][mois - 2] + smic_reference
            cumuls_employes[Numero_serie]['sum_pass_ajusted'][mois - 1] = cumuls_employes[Numero_serie]['sum_pass_ajusted'][mois - 2] + plafond_adjusted
        else:
            cumuls_employes[Numero_serie]['sum_remunerations'][mois - 1] = brut_mensuel_apres_absence
            cumuls_employes[Numero_serie]['sum_smic'][mois - 1] = smic_reference

            cumuls_employes[Numero_serie]['sum_pass_ajusted'][mois - 1] = plafond_adjusted




        brut_cum = cumuls_employes[Numero_serie]['sum_remunerations'][mois - 1]
        pass_cum = cumuls_employes[Numero_serie]['sum_pass_ajusted'][mois - 1]

        # Calcul des cotisations
        if mois > date_embauche_mois:
            sum_remunerations_mois = cumuls_employes[Numero_serie]['sum_remunerations'][mois - 1]



        else:
            sum_remunerations_mois = cumuls_employes[Numero_serie]['sum_remunerations'][mois - 1]




        base_tranche_a, base_tranche_b= calculate_base_pole_demploi(brut_cum,pass_cum,cum_base_tranche_a_prec_m1,cum_base_tranche_b_prec_m1,type_contrat,)
        taux_7C00,taux_7C10 = determiner_taux_pole_demploi(base_tranche_a,base_tranche_b)
        montant_tranche_a = base_tranche_a * taux_7C00 /100
        montant_tranche_b = base_tranche_b * taux_7C00 /100


        # AGS

        base_ags= calculate_base_AGS(brut_cum,pass_cum,cum_base_tranche_a_prec_m1,cum_base_tranche_b_prec_m1,type_contrat)
        taux_ags = determiner_taux_ags(base_ags)
        montant_7C20 = taux_ags * base_ags/100

        cum_base_tranche_a_prec_m2 = cum_base_tranche_a_prec_m1
        cum_base_tranche_a_prec_m1 += base_tranche_a

        cum_base_tranche_b_prec_m2 = cum_base_tranche_b_prec_m1
        cum_base_tranche_b_prec_m1 += base_tranche_b

        # Ajout des données au dataframe
        data.append({
            'Matricule': Numero_serie,
            'Statut de salariés': statut,
            'date de paie' : date_paie,
            'date embauche jour' : date_embauche_jour,
            'date embauche mois' : date_embauche_mois,
            'Effectif': effectif[i],
            'Statut de salariés': statut,
            'Type de contrat' : type_contrat,
            '1001 Montant Sal.': brut_mensuel,
            'Absences par Jour': jours_absence,
            'ABATTEMENT' : ABATTEMENT ,
            'Assiette Mois M (/102)' :  round(brut_mensuel_apres_absence,2),
            'Absences par Heure':heures_absence,
            'PLAFOND CUM' : cumuls_employes[Numero_serie]['sum_pass_ajusted'][mois - 1],

            # 'PLAFOND CUM Precedent' : round(cumuls_employes[Numero_serie]['sum_pass_ajusted'][mois - 2],2),
            'PLAFOND CUM M-1' : cumuls_employes[Numero_serie]['sum_pass_ajusted'][mois - 2],
            '4*PLAFOND' : 4* cumuls_employes[Numero_serie]['sum_pass_ajusted'][mois - 1],
            '4*PLAFOND CUM Precedent' : 4*cumuls_employes[Numero_serie]['sum_pass_ajusted'][mois - 2],





            "Cumul d'assiette ( Mois courant inclus) (/102)":cumuls_employes[Numero_serie]['sum_remunerations'][mois - 1],
            'ASSIETTE CUM PRECEDENTE':cumuls_employes[Numero_serie]['sum_remunerations'][mois - 2],
            'Rub 7C20' : '7C20',
            '7C20Base' : base_ags ,
            '7C20Taux 2' : taux_ags,
            '7C20Montant Pat.' : montant_7C20,


# base_tranche_a, base_tranche_b, taux_7C00,montant_tranche_a
            'Rub 7C00' : '7C00',
            '7C00Base' : base_tranche_a ,
            'Cum base 7C00' : cum_base_tranche_a_prec_m2,
            '7C00Taux 2' : taux_7C00,
            '7C00Montant Pat.' : montant_tranche_a,

            'Rub 7C10' : '7C10',
            '7C10Base' : base_tranche_b ,
            'Cum base 7C10' : cum_base_tranche_b_prec_m2,
            '7C10Taux 2' : taux_7C10,
            '7C10Montant Pat.' : montant_tranche_b,

            'Frontalier':frontaliers,
            'Region' : regions
        })



df = pd.DataFrame(data)






df = df.round(3)

print(df.head())
len(df)

# #supprimer les valeurs negatifs
# # Sélectionner les colonnes numériques
# numeric_df = df.select_dtypes(include=[np.number])
# # Identifier les lignes sans valeurs négatives
# rows_to_keep = ~(numeric_df < 0).any(axis=1)
# # Filtrer le DataFrame original pour ne garder que les lignes sans valeurs négatives
# df_cleaned = df[rows_to_keep]
# # Sauvegarder la base de données résultante en CSV
df.to_csv('pole.csv', index=False)

# 7C20Taux

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("pole.csv")

percentage = 0.18
def generer_taux_errone_maladie(eliminer_valeur=None):

    taux_errone = round(np.random.uniform(0, 10), 2)  
    while eliminer_valeur is not None and taux_errone == eliminer_valeur:
        taux_errone = round(np.random.uniform(0, 10), 2)  
    return taux_errone

maladie_rows = df.sample(frac=percentage).index
df['7C20 Fraud'] = 0
for idx in maladie_rows:
    if df.at[idx, '7C20Taux 2'] == 0.0:
        df.at[idx, '7C20Taux 2'] = generer_taux_errone_maladie(eliminer_valeur=0.0)
        df.at[idx, '7C20 Fraud'] = 1
    elif df.at[idx, '7C20Taux 2'] == 0.25:
        df.at[idx, '7C20Taux 2'] = generer_taux_errone_maladie(eliminer_valeur=0.25)
        df.at[idx, '7C20 Fraud'] = 1
    # elif df.at[idx, '7C20Base'] != 0.0:
    #     df.at[idx, '7C20Taux 2'] == 0
    #     df.at[idx, '7C20 Fraud'] = 1

df.to_csv("7C20_avec_anomalies.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


df = pd.read_csv('7C20_avec_anomalies.csv')
features = df[[ 'Type de contrat','7C20Base' ,'7C20Taux 2']]

features_encoded = pd.get_dummies(features, columns=['Type de contrat'], drop_first=True)


target = df['7C20 Fraud']

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


In [ ]:
import joblib
joblib.dump(rf_classifier, '7C20_new_model_random_forest.pkl')


In [ ]:
import pandas as pd
import joblib


rf_classifier_loaded = joblib.load('7C20_new_model_random_forest.pkl')

new_df = pd.read_csv('/content/test7c20 - pole (2).csv (3).csv')
df['7C20Taux 2'] = df['7C20Taux 2'].fillna(0)
df['7C20Base'] = df['7C20Base'].fillna(0)

features_new = new_df[[ 'Type de contrat','7C20Base' ,'7C20Taux 2']]

features_new_encoded = pd.get_dummies(features_new, columns=['Type de contrat'], drop_first=True)

missing_cols = set(features_encoded.columns) - set(features_new_encoded.columns)
for c in missing_cols:
    features_new_encoded[c] = 0

features_new_encoded = features_new_encoded[features_encoded.columns]

predictions = rf_classifier_loaded.predict(features_new_encoded)

new_df['anomalie_predite'] = predictions

# Sauvegarder la nouvelle base avec les prédictions
#new_df.to_csv('nouvelle_base_avec_predictions.csv', index=False)

print(predictions)


# 7C00 Taux

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv("pole.csv")


percentage = 0.20
def generer_taux_errone_maladie(eliminer_valeur=None):

    taux_errone = round(np.random.uniform(0, 10), 2) 
    while eliminer_valeur is not None and taux_errone == eliminer_valeur:
        taux_errone = round(np.random.uniform(0, 10), 2)  
    return taux_errone


maladie_rows = df.sample(frac=percentage).index
df['7C00 Fraud'] = 0
for idx in maladie_rows:
    if df.at[idx, '7C00Taux 2'] == 0.0:
        df.at[idx, '7C00Taux 2'] = generer_taux_errone_maladie(eliminer_valeur=0.0)
        df.at[idx, '7C00 Fraud'] = 1
    elif df.at[idx, '7C00Taux 2'] == 4.05:
        df.at[idx, '7C00Taux 2'] = generer_taux_errone_maladie(eliminer_valeur=4.05)
        df.at[idx, '7C00 Fraud'] = 1
    # elif df.at[idx, '7C00Base'] == 0.0:
    #     df.at[idx, '7C00Taux 2'] = round(np.random.uniform(1, 10), 2)
    #     df.at[idx, '7C00 Fraud'] = 1


df.to_csv("7C00_avec_anomalies.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


df = pd.read_csv('7C00_avec_anomalies.csv')

features = df[['7C00Base', '7C00Taux 2']]

# 3. Définir la colonne cible (7C00 Fraud)
target = df['7C00 Fraud']


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)


y_pred = rf_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)


print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

import joblib

# Exporter le modèle Random Forest entraîné
joblib.dump(rf_classifier, '7C00_new_model_random_forest.pkl')






In [ ]:
import pandas as pd
import joblib


rf_classifier_loaded = joblib.load('7C00_new_model_random_forest.pkl')


new_df = pd.read_csv('/content/test7c00 - pole (2).csv (3).csv')

new_df['7C00Taux 2'] = new_df['7C00Taux 2'].fillna(0)
new_df['7C00Base'] = new_df['7C00Base'].fillna(0)


features_new = new_df[['7C00Base', '7C00Taux 2']]


predictions = rf_classifier_loaded.predict(features_new)


new_df['anomalie_predite'] = predictions

# Optionnel : Sauvegarder la nouvelle base avec les prédictions
# new_df.to_csv('nouvelle_base_avec_predictions.csv', index=False)

print(predictions)


# 7C10

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv("pole.csv")
percentage = 0.20
def generer_taux_errone_maladie(eliminer_valeur=None):

    taux_errone = round(np.random.uniform(0, 10), 2) 
    while eliminer_valeur is not None and taux_errone == eliminer_valeur:
        taux_errone = round(np.random.uniform(0, 10), 2)  
    return taux_errone

maladie_rows = df.sample(frac=percentage).index
df['7C10 Fraud'] = 0
for idx in maladie_rows:
    if df.at[idx, '7C10Taux 2'] == 0.0:
        df.at[idx, '7C10Taux 2'] = generer_taux_errone_maladie(eliminer_valeur=0.0)
        df.at[idx, '7C10 Fraud'] = 1
    elif df.at[idx, '7C10Taux 2'] == 4.05:
        df.at[idx, '7C10Taux 2'] = generer_taux_errone_maladie(eliminer_valeur=4.05)
        df.at[idx, '7C10 Fraud'] = 1
    # elif df.at[idx, '7C10Base'] == 0.0:
    #     df.at[idx, '7C10Taux 2'] = round(np.random.uniform(1, 10), 2)
    #     df.at[idx, '7C10 Fraud'] = 1




# Sauvegarder le DataFrame modifié
df.to_csv("7C10_avec_anomalies.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


df = pd.read_csv('7C10_avec_anomalies.csv')

features = df[[ 'Type de contrat','7C10Base' ,'7C10Taux 2']]

features_encoded = pd.get_dummies(features, columns=['Type de contrat'], drop_first=True)

target = df['7C10 Fraud']

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators=110, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Afficher les résultats
print(f"Accuracy: {accuracy * 110:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


In [ ]:
import joblib

# Exporter le modèle Random Forest entraîné
joblib.dump(rf_classifier, '7C10_new_model_random_forest.pkl')


In [ ]:
import pandas as pd
import joblib

rf_classifier_loaded = joblib.load('7C10_new_model_random_forest.pkl')


new_df = pd.read_csv('/content/test7c10 - pole (2).csv (1).csv')
df['7C10Taux 2'] = df['7C10Taux 2'].fillna(0)
df['7C10Base'] = df['7C10Base'].fillna(0)


features_new = new_df[[ 'Type de contrat','7C10Base' ,'7C10Taux 2']]


features_new_encoded = pd.get_dummies(features_new, columns=['Type de contrat'], drop_first=True)

missing_cols = set(features_encoded.columns) - set(features_new_encoded.columns)
for c in missing_cols:
    features_new_encoded[c] = 0


features_new_encoded = features_new_encoded[features_encoded.columns]


predictions = rf_classifier_loaded.predict(features_new_encoded)


new_df['anomalie_predite'] = predictions



print(predictions)
